In [3]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import statsmodels.api as sm

data = load_boston()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target
X.head(3)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03


In [4]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.argmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))
                
         # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [5]:
result = stepwise_selection(X, y)

print('resulting features:')
print(result)           

Add                               3 with p-value 5.0811e-88


<ipython-input-4-c6da44f0c516>:24: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)


KeyError: "None of [Int64Index([3], dtype='int64')] are in the [columns]"

In [7]:
data = load_boston()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target
X['target'] = y
X

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


## stats feature selection 외국인 2015년 코드
+ https://planspace.org/20150423-forward_selection_with_statsmodels/

In [8]:
import statsmodels.formula.api as smf
def forward_selected(data, response):
    """Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    """
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response,
                                           ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(selected))
    model = smf.ols(formula, data).fit()
    return model
forward_selected(X, 'target')

In [9]:
model = forward_selected(X, 'target')
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                     128.2
Date:                Wed, 14 Oct 2020   Prob (F-statistic):          5.54e-137
Time:                        00:54:58   Log-Likelihood:                -1498.9
No. Observations:                 506   AIC:                             3022.
Df Residuals:                     494   BIC:                             3072.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     36.3411      5.067      7.171      0.000      26.385      46.298
LSTAT         -0.5226      0.047    -11.019      0.000      -0.616      -0.429
RM             3.8016      0.406      9.356      0.000       3.003       4.600
PTRATIO       -0.9465      0.129     -7.334      0.000      -1.200      -0.693
DIS           -1.4927      0.186     -8.037      0.000      -1.858      -1.128
NOX          -17.3760      3.535     -4.915      0.000     -24.322     -10.430
CHAS           2.7187      0.854      3.183      0.002       1.040       4.397
B              0.0093      0.003      3.475      0.001       0.004       0.015
ZN             0.0458      0.014      3.390      0.001       0.019       0.072
CRIM          -0.1084      0.033     -3.307      0.001      -0.173      -0.044
RAD            0.2996      0.063      4.726      0.000       0.175       0.424
TAX           -0.0118      0.003     -3.493      0.001      -0.018      -0.005
==============================================================================
Omnibus:                      178.430   Durbin-Watson:                   1.078
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              787.785
Skew:                           1.523   Prob(JB):                    8.60e-172
Kurtosis:                       8.300   Cond. No.                     1.47e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.47e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## 종합 github 2020-04-19
Best Subset Selection, Forward Stepwise, Backward Stepwise Classes in sk-learn style.  
This package is compatible to sklearn. Examples on Pipeline and GridSearchCV are given.  
https://github.com/xinhe97/StepwiseSelectionOLS  

In [ ]:
#### BackwardStepwiseOLS

In [10]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import random
import itertools
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.metrics import euclidean_distances

np.random.seed(2020)
random.seed(2020)

# my stepwise selection + sklearn style
class BackwardStepwiseOLS(BaseEstimator):

    def __init__(self, fK=3):
        self.fK = fK                        # number of predictors

    def myBic(self, n, mse, k):
        if k<=0:
            return np.nan
        else:
            return n*np.log(mse)+k*np.log(n)

    ################### Criteria ###################
    def processSubset(self, X,y,feature_index):
        # Fit model on feature_set and calculate rsq_adj
        regr = sm.OLS(y,X[:,feature_index]).fit()
        rsq_adj = regr.rsquared_adj
        bic = self.myBic(X.shape[0], regr.mse_resid, len(feature_index))
        rsq = regr.rsquared
        return {"model":regr, "rsq_adj":rsq_adj, "bic":bic, "rsq":rsq, "predictors_index":feature_index}

    ################### Backward Stepwise ###################
    def Backward(self,predictors_index,X,y):
        
        results = []
        for p in predictors_index:
            index_tmp = predictors_index.copy()
            index_tmp.remove(p)
            new_predictors_index = index_tmp
            new_predictors_index.sort()
            results.append(self.processSubset(X,y,new_predictors_index))
            # Wrap everything up in a nice dataframe
        models = pd.DataFrame(results)
        # Choose the model with the highest rsq_adj
        # best_model = models.loc[models['bic'].idxmin()]
        best_model = models.loc[models['rsq'].idxmax()]
        # Return the best model, along with model's other  information
        return best_model

    def BackwardK(self,X_est,y_est, fK):
        models_bwd = pd.DataFrame(columns=["model", "rsq_adj", "bic", "rsq", "predictors_index"])
        predictors_index = list(range(X_est.shape[1]))

        if X_est.shape[1]<=fK:
            print("use all predictors")
            best_model_bwd = self.processSubset(X_est,y_est,predictors_index)["model"]
            best_predictors = predictors_index
        else:
            i = X_est.shape[1]
            j = 0
            models_bwd.loc[j] = self.processSubset(X_est,y_est,predictors_index)
            i = i-1
            print(i)
            print(predictors_index)

            while i >= fK:
                j = j+1
                models_bwd.loc[j] = self.Backward(predictors_index,X_est,y_est)
                predictors_index = models_bwd.loc[j,'predictors_index']
                i = i-1
                print(i)
                print(predictors_index)

            print(models_bwd)
            best_model_bwd = models_bwd.loc[models_bwd['bic'].idxmin(),'model']
            # best_model_bwd = models_bwd.loc[models_bwd['rsq'].idxmax(),'model']
            best_predictors = models_bwd.loc[models_bwd['bic'].idxmin(),'predictors_index']
            # best_predictors = models_bwd.loc[models_bwd['rsq'].idxmax(),'predictors_index']
        return best_model_bwd, best_predictors


    def fit(self, X, y):
        X, y = check_X_y(X, y, accept_sparse=True)

        # hexin
        self.best_model_bwd, self.best_predictors = self.BackwardK(X,y,self.fK)
        # hexin

        self.is_fitted_ = True
        # `fit` should always return `self`
        return self

    def predict(self, X):
        X = check_array(X, accept_sparse=True)

        # hexin
        y_pred = self.best_model_bwd.predict(X[:,self.best_predictors])
        # hexin

        check_is_fitted(self, 'is_fitted_')
        return y_pred

    def get_params(self, deep=True):
        return {"fK": self.fK}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

    def score(self, X, y_true):
        return r2_score(y_true, self.predict(X))
        # return -mean_squared_error(y_true, self.predict(X))

if __name__ == '__main__':
    ###### DGP ######
    # Spare signals
    N = 1000
    P = 10 # Total number of inputs

    N_true_inputs = 5
    N_false_inputs = P - N_true_inputs
    n_obs = N/2
    n_pred = N/2
    error_sd = 1

    #True inputs have coefficient 1
    beta = np.matrix(np.zeros((P,1)))
    beta[:N_true_inputs,:] = 1

    #Simulate the data
    X = np.matrix(np.random.rand(N,P))
    epsilon = np.matrix(error_sd*np.random.normal(0,size=(N,1)))
    y = X*beta + epsilon

    # Pack the data into a dataframe
    DF = pd.concat([pd.DataFrame(X),pd.DataFrame(y)],axis=1)
    new_names_true = ['x_true_'+str(i) for i in range(1,N_true_inputs+1)]
    new_names_false = ['x_false_'+str(i) for i in range(1,N_false_inputs+1)]
    names = new_names_true + new_names_false + ['y']
    DF.columns = names

    # Now we split the data into an estimation and prediction sample. # Randomly draw n_obs observations
    train_index = random.sample(range(0,N),np.int(n_obs))
    train_index.sort()
    DF_estimation = DF.loc[train_index,:]
    DF_prediction = DF.drop(index=train_index)

    # ###### Algorithm ######
    # bwd = BackwardStepwiseOLS(fK=10)
    # bwd.fit(DF_estimation.drop('y',1), DF_estimation['y'])
    # bwd.predict(DF_prediction.drop('y',1))
    # print(bwd.score(DF_prediction.drop('y',1), DF_prediction['y']))

    ###### Algorithm ######
    bwd = BackwardStepwiseOLS(fK=1)
    bwd.fit(DF_estimation.drop('y',1), DF_estimation['y'])
    bwd.predict(DF_prediction.drop('y',1))
    print(bwd.score(DF_prediction.drop('y',1), DF_prediction['y']))

9
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
8
[0, 1, 2, 3, 4, 5, 6, 8, 9]
7
[0, 1, 2, 3, 4, 5, 8, 9]
6
[0, 1, 2, 3, 4, 8, 9]
5
[0, 1, 2, 3, 4, 9]
4
[0, 1, 2, 3, 4]
3
[0, 1, 2, 3]
2
[0, 1, 2]
1
[0, 1]
0
[0]
                                               model   rsq_adj         bic  \
0  <statsmodels.regression.linear_model.Regressio...  0.887225   33.274280   
1  <statsmodels.regression.linear_model.Regressio...  0.887454   26.041482   
2  <statsmodels.regression.linear_model.Regressio...  0.887638   19.011364   
3  <statsmodels.regression.linear_model.Regressio...  0.887793   12.106134   
4  <statsmodels.regression.linear_model.Regressio...  0.887932    5.271525   
5  <statsmodels.regression.linear_model.Regressio...  0.887450    1.202154   
6  <statsmodels.regression.linear_model.Regressio...  0.876483   41.478788   
7  <statsmodels.regression.linear_model.Regressio...  0.858160  104.422655   
8  <statsmodels.regression.linear_model.Regressio...  0.825279  202.454081   
9  <statsmodels.regressio

In [ ]:
#### ForwardStepwiseOLS

In [11]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import random
import itertools
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.metrics import euclidean_distances

np.random.seed(2020)
random.seed(2020)

# my stepwise selection + sklearn style
class ForwardStepwiseOLS(BaseEstimator):

    def __init__(self, fK=3):
        self.fK = fK                        # number of predictors

    def myBic(self, n, mse, k):
        if k<=0:
            return np.nan
        else:
            return n*np.log(mse)+k*np.log(n)

    ################### Criteria ###################
    def processSubset(self, X,y,feature_index):
        # Fit model on feature_set and calculate rsq_adj
        regr = sm.OLS(y,X[:,feature_index]).fit()
        rsq_adj = regr.rsquared_adj
        bic = self.myBic(X.shape[0], regr.mse_resid, len(feature_index))
        rsq = regr.rsquared
        return {"model":regr, "rsq_adj":rsq_adj, "bic":bic, "rsq":rsq, "predictors_index":feature_index}

    ################### Forward Stepwise ###################
    def forward(self,predictors_index,X,y):
        # Pull out predictors we still need to process
        remaining_predictors_index = [p for p in range(X.shape[1])
                                if p not in predictors_index]

        results = []
        for p in remaining_predictors_index:
            new_predictors_index = predictors_index+[p]
            new_predictors_index.sort()
            results.append(self.processSubset(X,y,new_predictors_index))
            # Wrap everything up in a nice dataframe
        models = pd.DataFrame(results)
        # Choose the model with the highest rsq_adj
        # best_model = models.loc[models['bic'].idxmin()]
        best_model = models.loc[models['rsq'].idxmax()]
        # Return the best model, along with model's other  information
        return best_model

    def forwardK(self,X_est,y_est, fK):
        models_fwd = pd.DataFrame(columns=["model", "rsq_adj", "bic", "rsq", "predictors_index"])
        predictors_index = []

        M = min(fK,X_est.shape[1])

        for i in range(1,M+1):
            print(i)
            models_fwd.loc[i] = self.forward(predictors_index,X_est,y_est)
            predictors_index = models_fwd.loc[i,'predictors_index']

        print(models_fwd)
        # best_model_fwd = models_fwd.loc[models_fwd['bic'].idxmin(),'model']
        best_model_fwd = models_fwd.loc[models_fwd['rsq'].idxmax(),'model']
        # best_predictors = models_fwd.loc[models_fwd['bic'].idxmin(),'predictors_index']
        best_predictors = models_fwd.loc[models_fwd['rsq'].idxmax(),'predictors_index']
        return best_model_fwd, best_predictors


    def fit(self, X, y):
        X, y = check_X_y(X, y, accept_sparse=True)

        # hexin
        self.best_model_fwd, self.best_predictors = self.forwardK(X,y,self.fK)
        # hexin

        self.is_fitted_ = True
        # `fit` should always return `self`
        return self

    def predict(self, X):
        X = check_array(X, accept_sparse=True)

        # hexin
        y_pred = self.best_model_fwd.predict(X[:,self.best_predictors])
        # hexin

        check_is_fitted(self, 'is_fitted_')
        return y_pred

    def get_params(self, deep=True):
        return {"fK": self.fK}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

    def score(self, X, y_true):
        return r2_score(y_true, self.predict(X))
        # return -mean_squared_error(y_true, self.predict(X))

if __name__ == '__main__':
    ###### DGP ######
    # Spare signals
    N = 1000
    P = 10 # Total number of inputs

    N_true_inputs = 5
    N_false_inputs = P - N_true_inputs
    n_obs = N/2
    n_pred = N/2
    error_sd = 1

    #True inputs have coefficient 1
    beta = np.matrix(np.zeros((P,1)))
    beta[:N_true_inputs,:] = 1

    #Simulate the data
    X = np.matrix(np.random.rand(N,P))
    epsilon = np.matrix(error_sd*np.random.normal(0,size=(N,1)))
    y = X*beta + epsilon

    # Pack the data into a dataframe
    DF = pd.concat([pd.DataFrame(X),pd.DataFrame(y)],axis=1)
    new_names_true = ['x_true_'+str(i) for i in range(1,N_true_inputs+1)]
    new_names_false = ['x_false_'+str(i) for i in range(1,N_false_inputs+1)]
    names = new_names_true + new_names_false + ['y']
    DF.columns = names

    # Now we split the data into an estimation and prediction sample. # Randomly draw n_obs observations
    train_index = random.sample(range(0,N),np.int(n_obs))
    train_index.sort()
    DF_estimation = DF.loc[train_index,:]
    DF_prediction = DF.drop(index=train_index)

    ###### Algorithm ######
    fwd = ForwardStepwiseOLS(fK=10)
    fwd.fit(DF_estimation.drop('y',1), DF_estimation['y'])
    fwd.predict(DF_prediction.drop('y',1))
    print(fwd.score(DF_prediction.drop('y',1), DF_prediction['y']))

1
2
3
4
5
6
7
8
9
10
                                                model   rsq_adj         bic  \
1   <statsmodels.regression.linear_model.Regressio...  0.739448  396.046314   
2   <statsmodels.regression.linear_model.Regressio...  0.825279  202.454081   
3   <statsmodels.regression.linear_model.Regressio...  0.859588   99.365348   
4   <statsmodels.regression.linear_model.Regressio...  0.876018   43.356333   
5   <statsmodels.regression.linear_model.Regressio...  0.887450    1.202154   
6   <statsmodels.regression.linear_model.Regressio...  0.887932    5.271525   
7   <statsmodels.regression.linear_model.Regressio...  0.887793   12.106134   
8   <statsmodels.regression.linear_model.Regressio...  0.887638   19.011364   
9   <statsmodels.regression.linear_model.Regressio...  0.887454   26.041482   
10  <statsmodels.regression.linear_model.Regressio...  0.887225   33.274280   

         rsq                predictors_index  
1   0.739969                             [0]  
2   0.825978   

In [ ]:
#### BestSubsetSelectionOLS

In [12]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import random
import itertools
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.metrics import euclidean_distances

np.random.seed(2020)
random.seed(2020)

# %%pixie_debugger
class BestSubsetSelectionOLS(BaseEstimator):
    def __init__ (self, fK=3):
        self.fK = fK              #number of predictors
        
    def myBic(self, n, mse, k):
        if k<=0:
            return np.nan
        else:
            return n*np.log(mse) + k*np.log(n)
        
    ############   Criteria   ##################################
    def processSubset(self, X,y,feature_set):
        regr = sm.OLS(y, X[list(feature_set)]).fit()

        rsq_adj = regr.rsquared_adj

        bic = self.myBic(X.shape[0], regr.mse_resid, len(feature_set))
        rsq = regr.rsquared
        return{"model": regr, "rsq_adj":rsq_adj, "bic":bic, "rsq":rsq, "best_predictors": feature_set}  
    
 ############## bext subset selection######################
    def getBest(self, X,y,fK):
        results = [] #fill results in a list
        #get X variable's all combinations(X.columns,k):
        X = pd.DataFrame(X)
        for combo in itertools.combinations(X.columns, fK):
            results.append(self.processSubset(X,y,combo))
        # Wrap everything up in a nice dataframe
        models = pd.DataFrame(results)
        # Choose the model with the highest rsq_adj
        best_model =  models.loc[models["rsq"].idxmax(), 'model']
        #Return best_model
        best_predictors =  models.loc[models["rsq"].idxmax(), 'best_predictors']
        print(fK, best_predictors)
        return best_model, best_predictors #later add feature_set
    
# forwardK should be not applicable to best selection since best only choose a specific fK

    def fit(self, X, y):
        X, y = check_X_y(X, y, accept_sparse = True)
        
        self.best_model, self.best_predictors = self.getBest(X,y, self.fK) #later add return predictors
        
        # hexin
        self.is_fitted_ = True
        # print(self.best_predictors)
        
        return self
    
    def predict(self, X):
        X = check_array(X, accept_sparse = True)
        
        # hexin
        y_pred = self.best_model.predict(X[:, list(self.best_predictors)]) #later add returning the feature_set
        # hexin
        
        check_is_fitted(self, 'is_fitted_')
        return y_pred
    
    def get_params(self, deep = True):
        return {"fK": self.fK}
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
    
    def score(self, X, y_true):
        return r2_score(y_true, self.predict(X))
 
if __name__ == '__main__': 
    N = 1000
    P = 10 # Total number of inputs

    N_true_inputs = 5
    N_false_inputs = P - N_true_inputs
    n_obs = N/2
    n_pred = N/2
    error_sd = 1

    # True inputs have coefficient 1
    beta = np.matrix(np.zeros((P,1)))
    beta[:N_true_inputs, :] = 1

    # stimulate the data
    X = np.matrix(np.random.rand(N,P))
    epsilon = np.matrix(error_sd*np.random.normal(0, size= (N,1)))
    y = X*beta + epsilon

    # Pack the data into a dataframe
    DF = pd.concat([pd.DataFrame(X), pd.DataFrame(y)], axis = 1)
    new_names_true = ['x_true_'+str(i) for i in range(1, N_true_inputs + 1)]
    new_names_false = ['x_true_' +str(i) for i in range (1, N_false_inputs +1)]
    names = new_names_true + new_names_false + ['y']
    DF.columns = names

    # Now we split the data into an estimation and prediction sample. # Randomly draw n_obs obervations
    train_index = random.sample(range(0,N), np.int(n_obs))
    train_index.sort()
    DF_estimation = DF.loc[train_index, :]
    DF_prediction = DF.drop(index = train_index) 

    ####### Algorithm ####################
    bfit = BestSubsetSelectionOLS(fK=5)
    bfit.fit(DF_estimation.drop('y', 1), DF_estimation['y'])
    bfit.predict(DF_prediction.drop('y',1))
    print(bfit.score(DF_prediction.drop('y',1), DF_prediction['y']))

5 (0, 1, 2, 3, 4)
0.27586697953903694


#### main

In [13]:
# from ForwardStepwiseOLS import *
# from BackwardStepwiseOLS import *
# from BestSubsetSelectionOLS import *


# DGP
# Spare signals
N = 1000
P = 10 # Total number of inputs

N_true_inputs = 5
N_false_inputs = P - N_true_inputs
n_obs = N/2
n_pred = N/2
error_sd = 1

#True inputs have coefficient 1
beta = np.matrix(np.zeros((P,1)))
beta[:N_true_inputs,:] = 1

#Simulate the data
X = np.matrix(np.random.rand(N,P))
epsilon = np.matrix(error_sd*np.random.normal(0,size=(N,1)))
y = X*beta + epsilon
y_label = y>1

# Pack the data into a dataframe
#DF = pd.concat([pd.DataFrame(X),pd.DataFrame(y)],axis=1)
DF = pd.concat([pd.DataFrame(X),pd.DataFrame(y_label)],axis=1)
new_names_true = ['x_true_'+str(i) for i in range(1,N_true_inputs+1)]
new_names_false = ['x_false_'+str(i) for i in range(1,N_false_inputs+1)]
names = new_names_true + new_names_false + ['y']
DF.columns = names

# Now we split the data into an estimation and prediction sample. # Randomly draw n_obs observations
train_index = random.sample(range(0,N),np.int(n_obs))
train_index.sort()
DF_estimation = DF.loc[train_index,:]
DF_prediction = DF.drop(index=train_index)

###### GridSearchCV ######

############
# # forward
# param_grid_pipe_fwd = {
#     'fwd__fK': [1,2,3,4,5,6,7,8,9,10]
# }
# pipe = Pipeline(steps=[('fwd', ForwardStepwiseOLS())])
# search = GridSearchCV(estimator=pipe, cv=5, param_grid=param_grid_pipe_fwd, n_jobs=-1)
# search.fit(DF_estimation.drop('y',1), DF_estimation['y'])

# print(search.best_params_)
# print(search.cv_results_)

############
# # backward
# # the hyperparameter fK: is the least number of features included
# param_grid_pipe_bwd = {
#     'bwd__fK': [1,2,3,4,5,6,7,8,9,10]
# }
# pipe = Pipeline(steps=[('bwd', BackwardStepwiseOLS())])
# search = GridSearchCV(estimator=pipe, cv=5, param_grid=param_grid_pipe_bwd, n_jobs=-1)
# search.fit(DF_estimation.drop('y',1), DF_estimation['y'])

# print(search.best_params_)
# print(search.cv_results_)

###########
# best subset
# the hyperparameter fK: is the number of features included
param_grid_pipe_bst = {
    'bst__fK': [1,2,3,4,5,6,7,8,9,10]
}
pipe = Pipeline(steps=[('bst', BestSubsetSelectionOLS())])
search = GridSearchCV(estimator=pipe, cv=5, param_grid=param_grid_pipe_bst, n_jobs=-1)
search.fit(DF_estimation.drop('y',1), DF_estimation['y'])

print(search.best_params_)
print(search.cv_results_)

9 (0, 1, 2, 3, 4, 5, 6, 7, 9)
{'bst__fK': 9}
{'mean_fit_time': array([0.0293222 , 0.08916249, 0.19556732, 0.3231153 , 0.40578017,
       0.34366755, 0.19995208, 0.08123059, 0.02187014, 0.00624886]), 'std_fit_time': array([0.00173853, 0.00499126, 0.01116728, 0.00813042, 0.01695689,
       0.01711219, 0.00624857, 0.00624897, 0.00765288, 0.00765325]), 'mean_score_time': array([0.00259314, 0.00259356, 0.00059853, 0.        , 0.00312395,
       0.        , 0.        , 0.00312419, 0.00624862, 0.00312443]), 'std_score_time': array([0.00048899, 0.00048827, 0.00119705, 0.        , 0.0062479 ,
       0.        , 0.        , 0.00624838, 0.00765296, 0.00624886]), 'param_bst__fK': masked_array(data=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'params': [{'bst__fK': 1}, {'bst__fK': 2}, {'bst__fK': 3}, {'bst__fK': 4}, {'bst__fK': 5}, {'bst__fK': 6}, {'bst_